In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import seaborn as sns
from keras.models import load_model
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
import shap
from scipy.stats import pearsonr
import sys
sys.path.insert(0, '/Users/huripari/Documents/PhD/TCs_Genesis/FS_TCG')
import utils_results as ut

In [2]:
project_dir = '/Users/huripari/Documents/PhD/TCs_Genesis'
fs_dir = os.path.join(project_dir, 'FS_TCG')
n_folds = 3
years = np.arange(1980, 2022, 1) # from 1980 to 2021 included
cluster_data_info = {'GLB': 'nc12', 'NEP': 'DSnc12', 'NWP': 'Anc10', 'NA': 'DSnc6', 'NI': 'DSnc12', 'SI':'DSnc9', 'SP': 'nc7'}
basin_names = {'GLB': 'Global', 'NEP': 'North East Pacific',  'NWP': 'North West Pacific', 'NA': 'North Atlantic',
              'NI': 'North Indian', 'SI': 'South Indian', 'SP': 'South Pacific'}

In [3]:
basins = ['GLB', 'NEP', 'NWP', 'NA', 'NI', 'SI', 'SP']
for b, basin in enumerate(basins):
    # Get the data for each basin
    cluster_data = cluster_data_info[basin]
    basin_name = basin_names[basin]
    run_name = f'selfeat80_top20_{cluster_data}_nv8_nd9'
    n_clusters = int(run_name.split('nc')[1].split('_')[0])
    nc_string = run_name.split('_')[2]
    # Load the file containing the time series of the predictors and of the target
    if 'DS' in nc_string:
        clusters_data = f'{basin}_{n_clusters}clusters_deseason'
    elif 'A' in nc_string:
        clusters_data = f'{basin}_{n_clusters}clusters_anomaly'
    else:
        clusters_data = f'{basin}_{n_clusters}clusters'
    cluster_data_dir = os.path.join(fs_dir, 'data', f'{clusters_data}')
    predictor_file = f'predictors_1970-2022_{n_clusters}clusters_8vars_9idxs.csv'
    predictors_df = pd.read_csv(os.path.join(cluster_data_dir, predictor_file), index_col=0)
    predictors_df.index = pd.to_datetime(predictors_df.index)
    # Get the run info and data
    run_info = ut.runs_info(basin, run_name)
    dataset_opt = run_info[0]
    dataset_opt_noFS = run_info[1]
    Y_pred = run_info[2]
    Y_pred_noFS = run_info[3]
    Y_test = run_info[4]
    X_test_eval = run_info[5]
    X_test_eval_noFS = run_info[6]
    mlps = run_info[7]
    mlps_noFS = run_info[8]
    perm_importance_mlp = run_info[9]
    perm_importance_mlp_noFS = run_info[10]
    shap_values_mlp = run_info[11]
    shap_values_mlp_noFS = run_info[12]
    X_test = pd.concat(X_test_eval)
    X_test_noFS = pd.concat(X_test_eval_noFS)
    # Load the seasonality if working with deseason data
    if "DS" in nc_string:
        cluster_data = f'{basin}_{n_clusters}clusters_deseason'
        data_dir = os.path.join(fs_dir, 'data', cluster_data)
        target_season = 'target_seasonality_1970-2022_2.5x2.5.csv'
        target_season_df = pd.read_csv(os.path.join(data_dir, target_season), index_col=0)
        target_season_df.index = pd.to_datetime(target_season_df.index)
    # Concat the predictions and the test values
    Y_pred_df = pd.concat(Y_pred)
    Y_test_df = pd.concat(Y_test)
    Y_pred_noFS_df = pd.concat(Y_pred_noFS)
    # Create df containing the deseasonalized time series
    Y_pred_df_noS = Y_pred_df.copy()
    Y_test_df_noS = Y_test_df.copy()
    Y_pred_noFS_df_noS = Y_pred_noFS_df.copy()
    # Deseasonalize the time series
    if "DS" in nc_string:
        Y_pred_df['tcg'] = Y_pred_df['tcg'] + target_season_df.loc[Y_pred_df.index, 'seasonal']
        Y_test_df = Y_test_df + target_season_df.loc[Y_test_df.index, 'seasonal']
        Y_pred_noFS_df['tcg'] = Y_pred_noFS_df['tcg'] + target_season_df.loc[Y_pred_noFS_df.index, 'seasonal']
        decomp_pred = seasonal_decompose(Y_pred_df, model='additive', period=12)
        decomp_test = seasonal_decompose(Y_test_df, model='additive', period=12)
        decomp_noFS = seasonal_decompose(Y_pred_noFS_df, model='additive', period=12)
    else:
        decomp_pred = seasonal_decompose(Y_pred_df, model='additive', period=12)
        decomp_test = seasonal_decompose(Y_test_df, model='additive', period=12)
        decomp_noFS = seasonal_decompose(Y_pred_noFS_df, model='additive', period=12)
        Y_pred_df_noS['tcg'] = Y_pred_df['tcg'] - decomp_pred.seasonal
        Y_test_df_noS = Y_test_df - decomp_test.seasonal
        Y_pred_noFS_df_noS['tcg'] = Y_pred_noFS_df['tcg'] - decomp_noFS.seasonal
    # Load time series of gpis
    gpis_path = os.path.join(fs_dir, 'data', f'{basin}_2.5x2.5_gpis_time_series.csv')
    gpis_df = pd.read_csv(gpis_path, index_col=0)
    gpis_df.index = pd.to_datetime(gpis_df.index)
    gpis_df = gpis_df.loc[Y_pred_df.index]
    engpi = gpis_df['engpi']
    ogpi = gpis_df['ogpi']
    # Deseasonalize the time series of gpis
    decomp_engpi = seasonal_decompose(engpi, model='additive')
    decomp_ogpi = seasonal_decompose(ogpi, model='additive')
    engpi_noS = engpi - decomp_engpi.seasonal
    ogpi_noS = ogpi - decomp_ogpi.seasonal
    # values smaller than 0 are set to 0
    Y_pred_df.loc[Y_pred_df['tcg'] < 0, 'tcg'] = 0.0
    Y_pred_noFS_df.loc[Y_pred_noFS_df['tcg'] < 0, 'tcg'] = 0.0
    # Compute seasonal mean values
    Y_pred_df_season = Y_pred_df.groupby(Y_pred_df.index.month).mean()
    Y_test_df_season = Y_test_df.groupby(Y_test_df.index.month).mean()
    Y_pred_noFS_df_season = Y_pred_noFS_df.groupby(Y_pred_noFS_df.index.month).mean()
    engpi_seasonal = engpi.groupby(engpi.index.month).mean()
    ogpi_seasonal = ogpi.groupby(ogpi.index.month).mean()
    # Compute annual cumulative values
    Y_pred_df_annual = Y_pred_df.groupby(Y_pred_df.index.year).sum()
    Y_test_df_annual = Y_test_df.groupby(Y_test_df.index.year).sum()
    Y_pred_noFS_df_annual = Y_pred_noFS_df.groupby(Y_pred_noFS_df.index.year).sum()
    engpi_annual = engpi.groupby(engpi.index.year).sum()
    ogpi_annual = ogpi.groupby(ogpi.index.year).sum()
    # Remove trend from annual time series
    Y_pred_df_noT = Y_pred_df.copy()
    Y_test_df_noT = Y_test_df.copy()
    Y_pred_noFS_df_noT = Y_pred_noFS_df.copy()
    Y_pred_df_noT['tcg'] = Y_pred_df['tcg'] - decomp_pred.trend
    Y_test_df_noT = Y_test_df - decomp_test.trend
    Y_pred_noFS_df_noT['tcg'] = Y_pred_noFS_df['tcg'] - decomp_noFS.trend
    engpi_noT = engpi - decomp_engpi.trend
    ogpi_noT = ogpi - decomp_ogpi.trend
    Y_pred_df_annual_noT = Y_pred_df_noT.groupby(Y_pred_df_noT.index.year).sum()
    Y_test_df_annual_noT = Y_test_df_noT.groupby(Y_test_df_noT.index.year).sum()
    Y_pred_noFS_df_annual_noT = Y_pred_noFS_df_noT.groupby(Y_pred_noFS_df_noT.index.year).sum()
    engpi_annual_noT = engpi_noT.groupby(engpi_noT.index.year).sum()
    ogpi_annual_noT = ogpi_noT.groupby(ogpi_noT.index.year).sum()
    # If global store the time series, if sub-basins sum the time series
    if basin == 'GLB':
        Y_pred_df_GLB = Y_pred_df
        Y_test_df_GLB = Y_test_df
        Y_pred_noFS_df_GLB = Y_pred_noFS_df
        engpi_GLB = engpi
        ogpi_GLB = ogpi
        Y_pred_df_noS_GLB = Y_pred_df_noS
        Y_test_df_noS_GLB = Y_test_df_noS
        Y_pred_noFS_df_noS_GLB = Y_pred_noFS_df_noS
        engpi_noS_GLB = engpi_noS
        ogpi_noS_GLB = ogpi_noS
        Y_pred_df_annual_GLB = Y_pred_df_annual
        Y_test_df_annual_GLB = Y_test_df_annual
        Y_pred_noFS_df_annual_GLB = Y_pred_noFS_df_annual
        engpi_annual_GLB = engpi_annual
        ogpi_annual_GLB = ogpi_annual
        Y_pred_df_annual_noT_GLB = Y_pred_df_annual_noT
        Y_test_df_annual_noT_GLB = Y_test_df_annual_noT
        Y_pred_noFS_df_annual_noT_GLB = Y_pred_noFS_df_annual_noT
        engpi_annual_noT_GLB = engpi_annual_noT
        ogpi_annual_noT_GLB = ogpi_annual_noT
    elif b == 1:
        Y_pred_df_SBS = Y_pred_df
        Y_test_df_SBS = Y_test_df
        Y_pred_noFS_df_SBS = Y_pred_noFS_df
        engpi_SBS = engpi
        ogpi_SBS = ogpi
        Y_pred_df_noS_SBS = Y_pred_df_noS
        Y_test_df_noS_SBS = Y_test_df_noS
        Y_pred_noFS_df_noS_SBS = Y_pred_noFS_df_noS
        engpi_noS_SBS = engpi_noS
        ogpi_noS_SBS = ogpi_noS
        Y_pred_df_annual_SBS = Y_pred_df_annual
        Y_test_df_annual_SBS = Y_test_df_annual
        Y_pred_noFS_df_annual_SBS = Y_pred_noFS_df_annual
        engpi_annual_SBS = engpi_annual
        ogpi_annual_SBS = ogpi_annual
        Y_pred_df_annual_noT_SBS = Y_pred_df_annual_noT
        Y_test_df_annual_noT_SBS = Y_test_df_annual_noT
        Y_pred_noFS_df_annual_noT_SBS = Y_pred_noFS_df_annual_noT
        engpi_annual_noT_SBS = engpi_annual_noT
        ogpi_annual_noT_SBS = ogpi_annual_noT
    else:
        Y_pred_df_SBS += Y_pred_df
        Y_test_df_SBS += Y_test_df
        Y_pred_noFS_df_SBS += Y_pred_noFS_df
        engpi_SBS += engpi
        ogpi_SBS += ogpi
        Y_pred_df_noS_SBS += Y_pred_df_noS
        Y_test_df_noS_SBS += Y_test_df_noS
        Y_pred_noFS_df_noS_SBS += Y_pred_noFS_df_noS
        engpi_noS_SBS += engpi_noS
        ogpi_noS_SBS += ogpi_noS
        Y_pred_df_annual_SBS += Y_pred_df_annual
        Y_test_df_annual_SBS += Y_test_df_annual
        Y_pred_noFS_df_annual_SBS += Y_pred_noFS_df_annual
        engpi_annual_SBS += engpi_annual
        ogpi_annual_SBS += ogpi_annual
        Y_pred_df_annual_noT_SBS += Y_pred_df_annual_noT
        Y_test_df_annual_noT_SBS += Y_test_df_annual_noT
        Y_pred_noFS_df_annual_noT_SBS += Y_pred_noFS_df_annual_noT
        engpi_annual_noT_SBS += engpi_annual_noT
        ogpi_annual_noT_SBS += ogpi_annual_noT

2025-03-18 11:30:18.153939: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-03-18 11:30:18.153977: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-03-18 11:30:18.153980: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-03-18 11:30:18.154017: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-18 11:30:18.154030: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-03-18 11:30:18.507810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [ ]:
# Compute the correlation coefficient and the MSE between the predictions and the test values
# Monthly
r_GLB, _ = pearsonr(Y_test_df_noS_GLB, Y_pred_df_noS_GLB['tcg']) # correllation compute with the deseasonalized values
r_noFS_GLB, _ = pearsonr(Y_test_df_noS_GLB, Y_pred_noFS_df_noS_GLB['tcg'])
r_engpi_GLB, _ = pearsonr(Y_test_df_noS_GLB, engpi_noS_GLB)
r_ogpi_GLB, _ = pearsonr(Y_test_df_noS_GLB, ogpi_noS_GLB)
mse_GLB = mean_squared_error(Y_test_df_GLB, Y_pred_df_GLB['tcg']) # mean squared error compute with the original values
mse_noFS_GLB = mean_squared_error(Y_test_df_GLB, Y_pred_noFS_df_GLB['tcg'])
mse_engpi_GLB = mean_squared_error(Y_test_df_GLB, engpi_GLB)
mse_ogpi_GLB = mean_squared_error(Y_test_df_GLB, ogpi_GLB)
# Annual
rY, _ = pearsonr(Y_test_df_annual_noT, Y_pred_df_annual_noT['tcg']) # correlation error compute with the detrended values
rY_noFS, _ = pearsonr(Y_test_df_annual_noT, Y_pred_noFS_df_annual_noT['tcg'])
rY_engpi, _ = pearsonr(Y_test_df_annual_noT, engpi_annual_noT)
rY_ogpi, _ = pearsonr(Y_test_df_annual_noT, ogpi_annual_noT)
mseY = mean_squared_error(Y_test_df_annual, Y_pred_df_annual['tcg']) # mean squared error compute with the original values
mseY_noFS = mean_squared_error(Y_test_df_annual, Y_pred_noFS_df_annual['tcg'])
mseY_engpi = mean_squared_error(Y_test_df_annual, engpi_annual)
mseY_ogpi = mean_squared_error(Y_test_df_annual, ogpi_annual)